## Skin Disease Detection using Transfer Learning (ResNet152)
- Dataset used: https://www.kaggle.com/datasets/ismailpromus/skin-diseases-image-dataset
- Initially trained on Kaggle (GPU: T4 x2)

In [ ]:
import kagglehub
import os
import numpy as np
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from sklearn.utils import class_weight
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.models import Model

Hello world


In [ ]:
#set seed to ensure constant results
SEED = 42  

random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
path = kagglehub.dataset_download("ismailpromus/skin-diseases-image-dataset")
print("Path to dataset files:", path)

In [ ]:
classes=sorted(os.listdir(os.path.join(path,'IMG_CLASSES')))

class_index={name:i for i,name in enumerate(classes)} #dict so its easy to convert label to index later
print(class_index, "\n")

In [ ]:

from tensorflow.keras.applications.resnet import preprocess_input
BATCH_SIZE=32
AUTOTUNE=tf.data.AUTOTUNE

def process_image(file_path, label): #process image by converting it into an array of 224x224 size and normalize it
    label = tf.cast(label, tf.int32)
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image=tf.image.resize(image,(224,224))
    image = preprocess_input(image)
    return image, label

def make_dataset(data): #data=[(file_path,label_str)]
  files= [f for f,_ in data]
  labels=[class_index[l] for _,l in data]
  ds=tf.data.Dataset.from_tensor_slices((files,labels))  #makes the tf dataset
  ds=ds.map(process_image,num_parallel_calls=AUTOTUNE)  #map() calls the function process_image to each element in the tf dataset
  ds=ds.shuffle(1000, seed=SEED).batch(BATCH_SIZE).prefetch(AUTOTUNE)
  return ds

In [ ]:

total_imgs=os.path.join(path, 'IMG_CLASSES')
train_data=[] #holds (file_path,class_name)
val_data=[]
test_data=[]
for classs in os.listdir(total_imgs):

    images=os.listdir(os.path.join(total_imgs,classs))
    train_num=int(0.8*len(images))
    validation_num=int(0.1*len(images))
    #these three lists hold the path_urls to the images within the class, 80% are testing rest 
    #are validation and test images
    train_imgs= random.sample(images,train_num)
    val_imgs=random.sample(list(set(images)-set(train_imgs)),validation_num)
    test_imgs= list((set(images)-set(train_imgs)) - set(val_imgs))

    for img_path in train_imgs:
        # Join total_imgs with the image filename to get the full path
        train_data.append((os.path.join(total_imgs, classs, img_path), classs))

    for img_path in val_imgs:
        # Join total_imgs with the image filename to get the full path
        val_data.append((os.path.join(total_imgs, classs, img_path), classs))
        
    for img_path in test_imgs:
        # Join total_imgs with the image filename to get the full path
        test_data.append((os.path.join(total_imgs, classs, img_path), classs))

random.shuffle(train_data)
random.shuffle(val_data)
random.shuffle(test_data)
train_ds=make_dataset(train_data)  #final tf.data.Dataset
val_ds=make_dataset(val_data)
test_ds=make_dataset(test_data)


print("Number of training images:", len(train_data))

In [ ]:
data_iterator=train_ds.as_numpy_iterator()  #we have to convert to np iterator so we can take stuff from the tf dataset
batch=data_iterator.next()
print(batch[0].shape) #batch[0] is the 32 images
print(batch[1].shape) #batch[1] is the 32 corresponding labels as a number

In [ ]:
train_labels = np.concatenate([y for x, y in train_ds], axis=0)
weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = dict(enumerate(weights))
print('Class weights: ')
print(class_weights)

## Model Building and Training.

In [ ]:
from tensorflow.keras.applications import ResNet152

base_model = ResNet152(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = True

for layer in base_model.layers[:-50]:
    layer.trainable = False

num_classes = 10
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.35)(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

model.summary()

In [ ]:
## Attempting with dropout(0.35), unfreezing 50 layers of resnet and lr tweaking with callback: ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', # The metric to watch
                                 factor=0.2,       # Reduce LR by factor of 5 (1 * 0.2)
                                 patience=2,       # Number of epochs with no improvement to wait
                                 min_lr=1e-6,      # Don't let the LR go below this value
                                 verbose=1)       # Print a message when LR is reduced


#optimizer = Adam(learning_rate=0.001) lr default

model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history=model.fit(train_ds,validation_data=val_ds,epochs=9,batch_size=BATCH_SIZE, callbacks=[lr_scheduler])
print(history)

In [ ]:
# Define the directory and filename for saving the model

# Saving the model in .h5 format.
dir_name = "enter_your_directory"
model_name = "enter_model_name.h5"

# Verify if directory exists, if not then create it
os.makedirs(dir_name, exist_ok=True)

# Save the model.
model.save(os.path.join(dir_name, model_name))

print(f"Model saved to {os.path.join(dir_name, model_name)}")

In [ ]:
# Saving the model in TFLITE format.

# Path to your saved Keras model (.h5 file)
h5_model_path = 'enter_your_director/enter_model_name.h5'

# Load the saved Keras model
model = tf.keras.models.load_model(h5_model_path)

# Directory where you want to save the .tflite model
tflite_dir = 'enter_your_directory/tflite_models'
os.makedirs(tflite_dir, exist_ok=True)

# Path for the converted .tflite file
tflite_path = os.path.join(tflite_dir, 'enter_model_name.tflite')

# Convert the Keras model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the converted model to the specified path
with open(tflite_path, 'wb') as f:
    f.write(tflite_model)

print(f"TFLite model saved at: {tflite_path}")
